__--outline--__

In [1]:
# Title
# from The National Renewable Energy Laboratory (NREL) Rooftop Energy Potential of Low Income Communities in America (REPLICA)

# Abstract


# Scope (notebook)


# Setup


# Obtain


# Scrub 


# Explore


# Model


# References

## Setup

In [2]:
import os
import numpy as np
import pandas as pd
import sqlite3

## Obtain

In [3]:
# listing files in the current directory
os.listdir()

['.git', '.gitignore', '.ipynb_checkpoints', 'data', 'README.md', 'sol.ipynb']

In [4]:
# previewing data directory
os.listdir('data/')

['.ipynb_checkpoints',
 'SEEDSII REPLICA Documentation.pdf',
 'seeds_ii_replica.csv']

Our dataset originates from:\ Mooney, Meghan; Sigrin, Ben (2018): Rooftop Energy Potential of Low Income Communities in America REPLICA. National Renewable Energy Laboratory. https://data.nrel.gov/submissions/81

The CSV flat file comprises a variety of sources.

We will instantiate the data as a Pandas dataframe and view the first rows.

In [5]:
data_df = pd.read_csv('data/seeds_ii_replica.csv', nrows=10)

# viewing first rows
data_df.head()

,geoid,gisjoin,state_fips,state_name,state_abbr,county_name,county_fips,tract_fips,area_km2,centroid_x,...,climate_zone_description,moisture_regime,locale,total_units,active_subsidies,avg_months_tenancy,fmr_2br,occ_rate,pct_eli_hh,lihtc_qualified
0,6001400100,G0600010400100,6,California,CA,Alameda County,1,400100,6.894336,-122.231947,...,Warm,C,City - Large,NaN,NaN,NaN,NaN,NaN,NaN,False
1,6001408100,G0600010408100,6,California,CA,Alameda County,1,408100,8.935563,-122.157212,...,Warm,C,City - Large,NaN,NaN,NaN,NaN,NaN,NaN,False
2,6001409000,G0600010409000,6,California,CA,Alameda County,1,409000,16.927907,-122.212192,...,Warm,C,City - Large,NaN,NaN,NaN,NaN,NaN,NaN,True
3,6001409900,G0600010409900,6,California,CA,Alameda County,1,409900,4.332550,-122.135826,...,Warm,C,City - Large,NaN,NaN,NaN,NaN,NaN,NaN,False
4,6001422000,G0600010422000,6,California,CA,Alameda County,1,422000,4.727481,-122.302847,...,Warm,C,City - Midsize,NaN,NaN,NaN,NaN,NaN,NaN,True


In [6]:
# viewing all attribute headers
for col in data_df.columns:
    print(col)

geoid
gisjoin
state_fips
state_name
state_abbr
county_name
county_fips
tract_fips
area_km2
centroid_x
very_low_mf_own_hh
very_low_mf_rent_hh
very_low_sf_own_hh
very_low_sf_rent_hh
low_mf_own_hh
low_mf_rent_hh
low_sf_own_hh
low_sf_rent_hh
mod_mf_own_hh
mod_mf_rent_hh
mod_sf_own_hh
mod_sf_rent_hh
mid_mf_own_hh
mid_mf_rent_hh
mid_sf_own_hh
mid_sf_rent_hh
high_mf_own_hh
high_mf_rent_hh
high_sf_own_hh
high_sf_rent_hh
very_low_mf_own_bldg_cnt
very_low_mf_rent_bldg_cnt
very_low_sf_own_bldg_cnt
very_low_sf_rent_bldg_cnt
low_mf_own_bldg_cnt
low_mf_rent_bldg_cnt
low_sf_own_bldg_cnt
low_sf_rent_bldg_cnt
mod_mf_own_bldg_cnt
mod_mf_rent_bldg_cnt
mod_sf_own_bldg_cnt
mod_sf_rent_bldg_cnt
mid_mf_own_bldg_cnt
mid_mf_rent_bldg_cnt
mid_sf_own_bldg_cnt
mid_sf_rent_bldg_cnt
high_mf_own_bldg_cnt
high_mf_rent_bldg_cnt
high_sf_own_bldg_cnt
high_sf_rent_bldg_cnt
very_low_mf_own_devp_cnt
very_low_mf_rent_devp_cnt
very_low_sf_own_devp_cnt
very_low_sf_rent_devp_cnt
low_mf_own_devp_cnt
low_mf_rent_devp_cnt
low_sf_

## Scrub

### Create SQLite3 Database

In [7]:
# create a new database file "conn":
conn = sqlite3.connect("data/rep.sqlite")

In [8]:
# load the CSV in chunks:
for row in pd.read_csv(
    "data/seeds_ii_replica.csv", chunksize=1000):
    # append each row to a new table "seed" in the database:
    row.to_sql("seed", conn, if_exists="append")

    # index on the 'geoid' column:
conn.execute("CREATE INDEX street ON seed(geoid)") 
conn.close()

### connect to sqlite database

In [9]:
# creating a connection to the database
conn = sqlite3.connect("data/rep.sqlite")

# instantiating a cursor
cur = conn.cursor()

### listing the sqlite table names

In [10]:
# get names of all tables in the sql database
sql_tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")

# assign table names to a variable as a list
table_list = list(map(lambda x: x[0], sql_tables.fetchall()))

# viewing/verifying table names
table_list

['seed']

## Explore

We can review our data description to select an area of investigation

### Custom Query

Let's find all of `data_df`'s column names for owner occupied properties that are listed as suitable buildings (those including the string "own_bldg_cnt").

In [12]:
# column names for owner-occupied, solar-suitable buildings
owner_suitable = [col for col in data_df.columns if "own_bldg_cnt" in col]
owner_suitable

['very_low_mf_own_bldg_cnt',
 'very_low_sf_own_bldg_cnt',
 'low_mf_own_bldg_cnt',
 'low_sf_own_bldg_cnt',
 'mod_mf_own_bldg_cnt',
 'mod_sf_own_bldg_cnt',
 'mid_mf_own_bldg_cnt',
 'mid_sf_own_bldg_cnt',
 'high_mf_own_bldg_cnt',
 'high_sf_own_bldg_cnt']

The columns listed in `owner_suitable` report the total annual solar power generation potential (MWh) for solar-suitable buildings.

Let's query these data for tracts with subsidies.

In [13]:
# creating a connection to the database
conn = sqlite3.connect("data/rep.sqlite")

# instantiating a cursor
cur = conn.cursor()

In [14]:
# querying attributes
cur.execute("""SELECT geoid, state_abbr, county_name, pop_total, 
                avg_monthly_bill_dlrs, dlrs_kwh,
                very_low_mf_own_bldg_cnt, very_low_sf_own_bldg_cnt, 
                low_mf_own_bldg_cnt, low_sf_own_bldg_cnt, 
                mod_mf_own_bldg_cnt, mod_sf_own_bldg_cnt, mid_mf_own_bldg_cnt, 
                mid_sf_own_bldg_cnt, high_mf_own_bldg_cnt, high_sf_own_bldg_cnt
               FROM seed
               WHERE active_subsidies > 0
            """);

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,geoid,state_abbr,county_name,pop_total,avg_monthly_bill_dlrs,dlrs_kwh,very_low_mf_own_bldg_cnt,very_low_sf_own_bldg_cnt,low_mf_own_bldg_cnt,low_sf_own_bldg_cnt,mod_mf_own_bldg_cnt,mod_sf_own_bldg_cnt,mid_mf_own_bldg_cnt,mid_sf_own_bldg_cnt,high_mf_own_bldg_cnt,high_sf_own_bldg_cnt
0,41017000500,OR,Deschutes County,5847,118.688180,0.088055,0.2681,85.9520,1.3405,67.1244,1.8767,130.1559,1.6086,172.7225,6.3077,698.2575
1,41017000800,OR,Deschutes County,6909,101.412973,0.110601,0.3102,165.8723,0.2979,104.7614,2.6930,239.5933,2.9068,278.3938,6.6598,633.4187
2,41017000900,OR,Deschutes County,7495,101.412973,0.110601,2.7129,341.7480,2.1251,205.4382,3.9131,267.7513,2.5207,294.0396,4.5255,355.3789
3,53033025006,WA,King County,6008,96.376683,0.111160,0.8993,21.1499,1.3617,28.1999,1.3257,35.2499,4.0874,100.2664,25.9989,785.6813
4,53033027701,WA,King County,5565,96.376683,0.111160,3.0984,69.5288,5.6797,100.4304,4.0178,117.4264,4.2618,149.8731,15.2504,444.2116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25690,45019002802,SC,Charleston County,2726,163.098857,0.145642,0.6422,21.5491,0.6421,14.9186,1.1227,53.8727,3.6657,96.1420,9.6551,307.4886
25691,27139080302,MN,Scott County,15090,83.731445,0.131830,5.3075,86.5339,10.6150,158.2085,13.9900,270.9649,38.4172,578.6411,114.7385,1812.8425
25692,34009021300,NJ,Cape May County,3957,123.070925,0.173899,5.3800,90.2319,8.2768,145.4222,12.5307,254.9269,11.8824,176.0835,17.5528,260.1831
25693,34023008900,NJ,Middlesex County,4004,97.396944,0.164645,3.2189,6.0137,7.5152,12.3071,5.2842,8.1404,8.1484,16.9270,14.0107,29.7838


In [15]:
# viewing info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25695 entries, 0 to 25694
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   geoid                     25695 non-null  int64  
 1   state_abbr                25695 non-null  object 
 2   county_name               25695 non-null  object 
 3   pop_total                 25695 non-null  int64  
 4   avg_monthly_bill_dlrs     25695 non-null  float64
 5   dlrs_kwh                  25695 non-null  float64
 6   very_low_mf_own_bldg_cnt  25695 non-null  float64
 7   very_low_sf_own_bldg_cnt  25695 non-null  float64
 8   low_mf_own_bldg_cnt       25695 non-null  float64
 9   low_sf_own_bldg_cnt       25695 non-null  float64
 10  mod_mf_own_bldg_cnt       25695 non-null  float64
 11  mod_sf_own_bldg_cnt       25695 non-null  float64
 12  mid_mf_own_bldg_cnt       25695 non-null  float64
 13  mid_sf_own_bldg_cnt       25695 non-null  float64
 14  high_m

In [16]:
# viewing summary statistics
df.describe()

,geoid,pop_total,avg_monthly_bill_dlrs,dlrs_kwh,very_low_mf_own_bldg_cnt,very_low_sf_own_bldg_cnt,low_mf_own_bldg_cnt,low_sf_own_bldg_cnt,mod_mf_own_bldg_cnt,mod_sf_own_bldg_cnt,mid_mf_own_bldg_cnt,mid_sf_own_bldg_cnt,high_mf_own_bldg_cnt,high_sf_own_bldg_cnt
count,2.569500e+04,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000,25695.000000
mean,3.301156e+10,4483.792683,111.248988,0.128887,2.252963,91.658702,1.814383,60.059984,2.930271,118.653164,3.884647,144.306591,8.936776,332.245147
std,1.276324e+10,2091.767106,22.411580,0.034146,3.286216,60.653672,2.573076,39.510449,3.680485,72.110110,5.132888,92.991432,13.069552,272.144669
min,1.000104e+10,0.000000,46.845787,0.029553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.206250e+10,3031.000000,97.396944,0.107758,0.554200,48.177950,0.287400,32.391250,0.717850,67.934250,0.834150,78.252900,1.654650,133.461350
50%,3.403903e+10,4206.000000,111.794393,0.118972,1.323000,81.199500,1.002100,53.732200,1.850800,111.936400,2.179700,134.853400,4.339100,285.136500
75%,4.210101e+10,5593.000000,127.104540,0.138892,2.804850,123.687550,2.357900,80.739650,3.880500,159.741800,4.998850,196.115200,10.673550,462.297350
max,5.604595e+10,35527.000000,262.413841,0.340253,73.180300,750.564300,56.182500,457.833300,100.469100,791.294500,99.887400,1363.645500,218.978500,3964.257500


In [17]:
# # exiting database connection    
conn.close()